In [1]:
%load_ext autoreload
%autoreload 2

from kaggle_speech_recog import *

/usr/local/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/usr/local/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Get feature and label matrices

In [2]:
X_vector_len = 16000
train = SpeechList.get_train('train/audio')
XY_train_valid = train.get_group_unknown_spectrogram_X_and_Y(X_vector_len=X_vector_len, spec_v='3', take_log=False, 
                                                             split_noise=True, n_fabricate_noise=1600, 
                                                             split=0.85)

# XY_train_valid = (X_train_known, X_train_unknown, Y_train_known, Y_train_unknown, X_valid, Y_valid)

/usr/local/anaconda3/lib/python3.6/site-packages/scipy/io/wavfile.py:273: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)


SKIPPED train/audio/_background_noise_/README.md File format b'# Ba'... not understood.


/usr/local/anaconda3/lib/python3.6/site-packages/librosa/filters.py:261: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  warnings.warn('Empty filters detected in mel frequency basis. '


# Get logits via each model

In [3]:
logs = ['SpectrogramMultiLSTMRandomInputModify_graph_03_lr_decay_run_01_group_unknown.log', 
        'NoveltyDetectionSpectrogramMultiLSTMRandomInputModify_graph_01_run_01.log',
        'SpectrogramMultiLSTMRandomInputModify_graph_02_run_01.log',
        'SpectrogramMultiLSTMRandomInputModify2_graph_02_lower_lr_lr_decay_run_01_group_unknown.log',  # switched from 10
        #'SpectrogramMultiLSTMRandomInputModify_graph_03_finer_layers_run_01.log',  # covered by 5
        #'SpectrogramMultiLSTMRandomInputModify_graph_04_kindof_finer_layers_run_01.log',  # covered by 5
        'SpectrogramCLSTMBidirectional_graph_03_run_01.log', 
        'SpectrogramDSConvDropout_graph_01_run_01.log',
        'SpectrogramConcatCLSTM2_graph_01_run_01_group_unknown.log',
        'SpectrogramConcatCLSTM_graph_03_conv3x3_lower_lr_run_01_group_unknown.log',
        #'SpectrogramConcatCLSTM_graph_01_run_01.log',  kind of covered by 19
        'SpectrogramConcatCLSTM_graph_04_conv5x1_lower_lr_run_01_group_unknown.log']

X_train_known, X_train_unknown, Y_train_known, Y_train_unknown, X_valid, Y_valid = XY_train_valid

L_train_known = gather_logits(X_train_known, log_dir='logs', logs=logs)
L_train_unknown = gather_logits(X_train_unknown, log_dir='logs', logs=logs)
L_valid = gather_logits(X_valid, log_dir='logs', logs=logs)

XY_train_valid = (L_train_known, L_train_unknown, Y_train_known, Y_train_unknown, L_valid, Y_valid)

INFO:tensorflow:Restoring parameters from checkpoints/SpectrogramMultiLSTMRandomInputModify_graph_03_lr_decay_run_01_group_unknown/best/model-57354
INFO:tensorflow:Restoring parameters from checkpoints/NoveltyDetectionSpectrogramMultiLSTMRandomInputModify_graph_01_run_01/best/model-82750
INFO:tensorflow:Restoring parameters from checkpoints/SpectrogramMultiLSTMRandomInputModify_graph_02_run_01/best/model-90650
INFO:tensorflow:Restoring parameters from checkpoints/SpectrogramMultiLSTMRandomInputModify2_graph_02_lower_lr_lr_decay_run_01_group_unknown/best/model-53088
INFO:tensorflow:Restoring parameters from checkpoints/SpectrogramCLSTMBidirectional_graph_03_run_01/best/model-17450
INFO:tensorflow:Restoring parameters from checkpoints/SpectrogramDSConvDropout_graph_01_run_01/best/model-22740
INFO:tensorflow:Restoring parameters from checkpoints/SpectrogramConcatCLSTM2_graph_01_run_01_group_unknown/best/model-23858
INFO:tensorflow:Restoring parameters from checkpoints/SpectrogramConcatCLS

# Train

In [8]:
graph_name = 'Ensemble'  # Specify graph class name

g_cnfg = Config('graph_04_9models')  # Specify graph configuration name
g_cnfg.X_n_features = XY_train_valid[0].shape[1]
g_cnfg.Y_vector_len = XY_train_valid[-1].shape[1]

g_cnfg.flat_hiddens = [256, 256]

g_cnfg.lr_initial = 0.001
g_cnfg.lr_decay_steps = 100000000
g_cnfg.lr_decay_rate = 1.0  # 1.0 means no decay

t_cnfg = Config('run_01')  # Specify train configuration name
t_cnfg.ensemble_logs = logs
t_cnfg.ckp_dir = 'checkpoints'
t_cnfg.tb_dir = 'tensorboard'
t_cnfg.log_dir = 'logs'
t_cnfg.le = train.le
t_cnfg.random_pick_unknown = True
t_cnfg.max_step = 100000000
t_cnfg.batch_size = 150
t_cnfg.dropout_keep_prob = 0.5
t_cnfg.log_every = int(int(XY_train_valid[0].shape[0] / 11 * 12) / t_cnfg.batch_size / 2)
t_cnfg.n_ave_ll_valid = 8
t_cnfg.start_step_early_stopping = 100000000
t_cnfg.early_stopping_patience =0.1

In [9]:
print(g_cnfg)
print(t_cnfg)

GraphClass = globals()[graph_name]
graph = GraphClass(g_cnfg)
graph.train_model(t_cnfg, XY_train_valid, annotate=False)

name: graph_04_9models
X_n_features: 154
Y_vector_len: 12
flat_hiddens: [256, 256]
lr_decay_rate: 1.0
lr_decay_steps: 100,000,000
lr_initial: 0.001

name: run_01
batch_size: 150
ckp_dir: checkpoints
dropout_keep_prob: 0.5
early_stopping_patience: 0.1
ensemble_logs: ['SpectrogramMultiLSTMRandomInputModify_graph_03_lr_decay_run_01_group_unknown.log', 'NoveltyDetectionSpectrogramMultiLSTMRandomInputModify_graph_01_run_01.log', 'SpectrogramMultiLSTMRandomInputModify_graph_02_run_01.log', 'SpectrogramMultiLSTMRandomInputModify2_graph_02_lower_lr_lr_decay_run_01_group_unknown.log', 'SpectrogramCLSTMBidirectional_graph_03_run_01.log', 'SpectrogramDSConvDropout_graph_01_run_01.log', 'SpectrogramConcatCLSTM2_graph_01_run_01_group_unknown.log', 'SpectrogramConcatCLSTM_graph_03_conv3x3_lower_lr_run_01_group_unknown.log', 'SpectrogramConcatCLSTM_graph_04_conv5x1_lower_lr_run_01_group_unknown.log']
le: LabelEncoder()
log_dir: logs
log_every: 79
max_step: 100,000,000
n_ave_ll_valid: 8
random_pick_un

KeyboardInterrupt: 